In [0]:
from pyspark.sql.functions import col, trim, when

df_bronze = spark.table("churn_catalog.raw.customer_data")

df_silver = (
    df_bronze

    # Remove duplicates
    .dropDuplicates(["customer_id"])

    # Remove rows where customer_id is null
    .filter(col("customer_id").isNotNull())

    # Clean string columns
    .withColumn("gender", trim(col("gender")))
    .withColumn("country", trim(col("country")))

    # Standardize gender values
    .withColumn("gender",
        when(col("gender").isin("Male", "male", "M"), "Male")
        .when(col("gender").isin("Female", "female", "F"), "Female")
        .otherwise("Unknown")
    )

    # Remove invalid ages
    .filter((col("age") > 0) & (col("age") < 80))

    # Replace negative balance & salary with 0
    .withColumn("balance", when(col("balance") < 0, 0).otherwise(col("balance")))
    .withColumn("estimated_salary", when(col("estimated_salary") < 0, 0).otherwise(col("estimated_salary")))

    # Convert credit_card (Yes/No or 1/0)
    .withColumn("credit_card",
        when(col("credit_card") == 1, 1)
        .otherwise(0)
    )

    # Convert active_member (Yes/No or 1/0)
    .withColumn("active_member",
        when(col("active_member") == 1, 1)
        .otherwise(0)
    )
)

display(df_silver)

df_silver.write.format("delta").mode("overwrite").saveAsTable(
    "churn_catalog.processed.customer_profiles"
)


# SILVER LAYER SUMMARY


In [0]:
from pyspark.sql.functions import col, when, count

print("SILVER LAYER SUMMARY")

# Total rows
print("Total rows after cleaning:", df_silver.count())

# Distinct customer IDs
print("Distinct customer_id:", df_silver.select("customer_id").distinct().count())

# Null count for each column
null_summary = df_silver.select([
    count(when(col(c).isNull(), c)).alias(c) for c in df_silver.columns
])
print("Null Value Summary:")
display(null_summary)

# Preview final cleaned data
print("Sample Cleaned Silver Data:")
display(df_silver.limit(10))
